# cross-validation analysis
5-fold cross-validation to ensure our results not overfitted

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
np.random.seed(42)

In [ ]:
# Load dataset
data = pd.read_csv("wdbc.data", header=None)

data.columns = [
    "id", "diagnosis", "radius1", "texture1", "perimeter1", "area1", 
    "smoothness1", "compactness1", "concavity1", "concave_points1", 
    "symmetry1", "fractal_dimension1", "radius2", "texture2", 
    "perimeter2", "area2", "smoothness2", "compactness2", "concavity2", 
    "concave_points2", "symmetry2", "fractal_dimension2", "radius3", 
    "texture3", "perimeter3", "area3", "smoothness3", "compactness3", 
    "concavity3", "concave_points3", "symmetry3", "fractal_dimension3"
]

X = data.drop(['id', 'diagnosis'], axis=1)
y = (data['diagnosis'] == 'M').astype(int)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Dataset: {X.shape[0]} samples, {X.shape[1]} features")

## Run 5-Fold Cross-Validation

In [ ]:
# Setup cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}

# Logistic Regression
print("Running Logistic Regression cross-validation...")
logreg = LogisticRegression(max_iter=10000, random_state=42)
cv_lr = cross_validate(logreg, X_scaled, y, cv=cv, scoring=scoring, return_train_score=True)

# Neural Network
print("Running Neural Network cross-validation...")
nn = MLPClassifier(hidden_layer_sizes=(16,), activation='relu', max_iter=1000, random_state=42)
cv_nn = cross_validate(nn, X_scaled, y, cv=cv, scoring=scoring, return_train_score=True)

print("\n Complete")

## Results Summary

In [ ]:
print("CROSS-VALIDATION RESULTS (5 Folds)")
print("="*70)

print("\nLogistic Regression:")
print(f"  Accuracy:  {cv_lr['test_accuracy'].mean():.4f} ± {cv_lr['test_accuracy'].std():.4f}")
print(f"  Precision: {cv_lr['test_precision'].mean():.4f} ± {cv_lr['test_precision'].std():.4f}")
print(f"  Recall:    {cv_lr['test_recall'].mean():.4f} ± {cv_lr['test_recall'].std():.4f}")
print(f"  F1-Score:  {cv_lr['test_f1'].mean():.4f} ± {cv_lr['test_f1'].std():.4f}")

print("\nNeural Network:")
print(f"  Accuracy:  {cv_nn['test_accuracy'].mean():.4f} ± {cv_nn['test_accuracy'].std():.4f}")
print(f"  Precision: {cv_nn['test_precision'].mean():.4f} ± {cv_nn['test_precision'].std():.4f}")
print(f"  Recall:    {cv_nn['test_recall'].mean():.4f} ± {cv_nn['test_recall'].std():.4f}")
print(f"  F1-Score:  {cv_nn['test_f1'].mean():.4f} ± {cv_nn['test_f1'].std():.4f}")

print("\n" + "="*70)

## Overfitting Check

In [ ]:
lr_gap = cv_lr['train_accuracy'].mean() - cv_lr['test_accuracy'].mean()
nn_gap = cv_nn['train_accuracy'].mean() - cv_nn['test_accuracy'].mean()

print("\nOVERFITTING ANALYSIS:")
print(f"Logistic Regression - Train vs Test gap: {lr_gap:.4f}")
print(f"Neural Network - Train vs Test gap:      {nn_gap:.4f}")
print("\nInterpretation:")
print("  Gap < 0.05 = No overfitting")
print("  Gap 0.05-0.10 = Minor overfitting")
print("  Gap > 0.10 = Overfitting")
print("\nResult:")
if lr_gap < 0.05 and nn_gap < 0.05:
    print("  Both models show no signs of overfitting")
elif lr_gap < 0.10 and nn_gap < 0.10:
    print("  Minor overfitting detected, but within acceptable range")
else:
    print("  Overfittingn detected")

## Graph 1

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Cross-Validation Results Across 5 Folds', fontsize=16, fontweight='bold')

metrics = ['accuracy', 'precision', 'recall', 'f1']

for idx, metric in enumerate(metrics):
    row = idx // 2
    col = idx % 2
    ax = axes[row, col]
    
    data_to_plot = [
        cv_lr[f'test_{metric}'],
        cv_nn[f'test_{metric}']
    ]
    
    bp = ax.boxplot(data_to_plot, labels=['Logistic Regression', 'Neural Network'],
                    patch_artist=True, widths=0.6)
    
    bp['boxes'][0].set_facecolor('#3498db')
    bp['boxes'][1].set_facecolor('#e74c3c')
    
    ax.set_title(f'{metric.capitalize()}', fontsize=12, fontweight='bold')
    ax.set_ylabel('Score', fontsize=10)
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim([0.90, 1.0])
    
    means = [np.mean(d) for d in data_to_plot]
    ax.plot([1, 2], means, 'D', color='gold', markersize=8, label='Mean')
    ax.tick_params(axis='x', rotation=15)

plt.tight_layout()
plt.savefig('figures/cross_validation_results.png', dpi=300, bbox_inches='tight')
plt.show()

## Graph 2

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Training vs Test Performance', fontsize=16, fontweight='bold')

models = [('Logistic Regression', cv_lr, '#3498db'), ('Neural Network', cv_nn, '#e74c3c')]

for idx, (name, results, color) in enumerate(models):
    ax = axes[idx]
    
    train_means = [results[f'train_{m}'].mean() for m in metrics]
    test_means = [results[f'test_{m}'].mean() for m in metrics]
    test_stds = [results[f'test_{m}'].std() for m in metrics]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    ax.bar(x - width/2, train_means, width, label='Train', color=color, alpha=0.5)
    ax.bar(x + width/2, test_means, width, label='Test (CV)', color=color, alpha=1.0)
    ax.errorbar(x + width/2, test_means, yerr=test_stds, fmt='none', color='black', capsize=5)
    
    ax.set_xlabel('Metrics', fontsize=12, fontweight='bold')
    ax.set_ylabel('Score', fontsize=12, fontweight='bold')
    ax.set_title(name, fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels([m.capitalize() for m in metrics])
    ax.legend()
    ax.set_ylim([0.90, 1.0])
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('figures/overfitting_check.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print("CONCLUSION")
print("="*70)
print("\n Models tested on 5 independent test sets")
print(" Low standard deviation shows consistent performance")
print(f" Train-test gaps < 0.05 indicate no overfitting")
print(" Neural Network maintains advantage over Logistic Regression")
print("\n Confirms our original results are ok")
print("="*70)